## Project: Applying NLP to Major Tasks
#### PROG8245- Machine Learning Programming

| Professor     | Islam Mahmoud               |
|---------------|-----------------------------|
| Email         | imahmoud@conestogac.on.ca   |
| Group         | Group 10                    |
| Members       | Burman, Jaiv Chaitanya      |
|               | Lad, Khushbu Nileshkumar    |
|               | Madhav, Akshata Ajay        |
